In [4]:
import numpy as np
import pandas as pd
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
sys.path.append(str(PROJECT_ROOT))

from src.markov_chain import (
    create_user_journeys,
    create_transition_matrix,
    extract_q_matrix,
    extract_r_matrix,
    compute_fundamental_matrix,
    removal_effects
)

from src.shapley import preprocess_for_shapley, compute_shapley


In [8]:
ga = pd.read_csv(r"C:\Users\Krina\atlas\data\ga_2.csv")

C:\Users\Krina\AppData\Local\Temp\ipykernel_22824\2394555535.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  ga = pd.read_csv(r"C:\Users\Krina\atlas\data\ga_2.csv")


In [16]:
paths = create_user_journeys(ga)
print(paths)

                    user_id  \
0            62349695125717   
1           170187170673177   
2           197671390269035   
3           277260039178149   
4           293088006393965   
...                     ...   
165537  9999594748540438502   
165538  9999773079368714197   
165539  9999799804624735268   
165540  9999825631375939274   
165541  9999944980984230465   

                                             session_time  \
0                               [2017-06-07 06:34:00 UTC]   
1       [2017-06-08 18:28:30 UTC, 2017-06-08 21:18:32 ...   
2                               [2017-05-03 00:54:30 UTC]   
3                               [2017-07-15 08:06:37 UTC]   
4       [2017-07-16 23:10:40 UTC, 2017-07-17 08:56:29 ...   
...                                                   ...   
165537  [2017-06-03 16:43:51 UTC, 2017-07-05 13:56:33 ...   
165538                          [2017-06-11 19:43:43 UTC]   
165539                          [2017-06-11 20:25:17 UTC]   
165540           

In [10]:
# Transition matrices
count_df, prob_df = create_transition_matrix(ga, paths)

In [11]:
# Build baseline N and R
absorbing = ["conversion", "dropped"]  # keep consistent with your GA encoding
Q, transient_states = extract_q_matrix(prob_df, absorbing_states=absorbing)
R, _ = extract_r_matrix(prob_df, absorbing_states=absorbing)

N = compute_fundamental_matrix(Q)


In [12]:
# Markov removal effects (Series)
markov_removal = removal_effects(prob_df, absorbing_states=absorbing, baseline_N=N, R_matrix=R)


In [13]:
# Shapley % (Series)
shap_df = preprocess_for_shapley(paths)
shapley_pct = compute_shapley(shap_df)

In [14]:
markov_removal.head()

referral          0.80
organic search    0.51
direct            0.31
paid search       0.09
social            0.03
dtype: float64

In [15]:
shapley_pct.head()

organic search    56.11
direct            18.65
referral          11.29
social             7.96
paid search        3.33
dtype: float64